In [19]:
import requests
import json

username = "yao"  # Replace with your username
password = "yaojianzhu"  # Replace with your password
problem_id = 4  # Replace with the actual problem ID you want to submit to
web_ip = "http://cnyaojz.com:25568"
compiler = "cc.cc11"  # Replace with the desired language key, e.g., "cc.cc11" for C++11, see lang.json
your_code = """
#include <bits/stdc++.h>
using namespace std;
int a[1005][1005];
int main (){
	int n,dir=0;
	scanf ("%d",&n);
	int dx[4]={1,0,-1,0},x=1;
	int dy[4]={0,-1,0,1},y=n;
	for (int i=1;i<=n*n;i++){
		a[x][y]=i;
		x+=dx[dir];
		y+=dy[dir];
		if (x<=0||x>n||y<=0||y>n||a[x][y]){
			x-=dx[dir];y-=dy[dir];
			dir=(dir+1)%4;
			x+=dx[dir];y+=dy[dir];
		}
	}
	for (int i=1;i<=n;i++){
		for (int j=1;j<=n;j++){
			printf ("%d ",a[i][j]);
		}
		puts("");
	}
	return 0;
}
""" # Replace with your actual code

# Step 1: Log in to get the session cookie
login_url = f"{web_ip}/login"
login_data = {
	"uname": username,
	"password": password,
	"rememberme": True
}
session = requests.Session()
login_response = session.post(login_url, json=login_data)

if login_response.status_code != 200:
	print("Login failed:", login_response.text)
	exit()
import regex
# Regex pattern for recursive matching
json_pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}', regex.DOTALL)
# export enum STATUS {
#     STATUS_WAITING = 0,
#     STATUS_ACCEPTED = 1,
#     STATUS_WRONG_ANSWER = 2,
#     STATUS_TIME_LIMIT_EXCEEDED = 3,
#     STATUS_MEMORY_LIMIT_EXCEEDED = 4,
#     STATUS_OUTPUT_LIMIT_EXCEEDED = 5,
#     STATUS_RUNTIME_ERROR = 6,
#     STATUS_COMPILE_ERROR = 7,
#     STATUS_SYSTEM_ERROR = 8,
#     STATUS_CANCELED = 9,
#     STATUS_ETC = 10,
#     STATUS_HACKED = 11,
#     STATUS_JUDGING = 20,
#     STATUS_COMPILING = 21,
#     STATUS_FETCHED = 22,
#     STATUS_IGNORED = 30,
#     STATUS_FORMAT_ERROR = 31,
#     STATUS_HACK_SUCCESSFUL = 32,
#     STATUS_HACK_UNSUCCESSFUL = 33,
# }
code_to_status = {
	0: "Waiting",
	1: "Accepted",
	2: "Wrong Answer",
	3: "Time Limit Exceeded",
	4: "Memory Limit Exceeded",
	5: "Output Limit Exceeded",
	6: "Runtime Error",
	7: "Compile Error",
	8: "System Error",
	9: "Canceled",
	10: "ETC",
	11: "Hacked",
	20: "Judging",
	21: "Compiling",
	22: "Fetched",
	30: "Ignored",
	31: "Format Error",
	32: "Hack Successful",
	33: "Hack Unsuccessful"
}

def append_user_message(player_messages, player_store_message, user_message):
	# check the role of the last message
	if len(player_messages) == 0 or player_messages[-1]["role"] == "assistant":
		player_messages.append({
			"role": "user",
			"content": user_message
		})
		player_store_message.append({
			"role": "user",
			"content": user_message
		})
	elif player_messages[-1]["role"] == "user":
		player_messages[-1]["content"] += "\n" + user_message
		player_store_message[-1]["content"] += "\n" + user_message

def append_assistant_message(player_messages, player_store_message, assistant_message):
	player_messages.append({
		"role": "assistant",
		"content": assistant_message
	})
	player_store_message.append({
		"role": "assistant",
		"content": assistant_message
	})


In [20]:
def parse_problem_text(pdoc):
	content = pdoc['content']
	title = pdoc['title']
	tag = pdoc['tag']
	config = pdoc['config']
	problem_text = "Please solve the following problem:\n\n"
	problem_text += f"Title: {title}\n"
	problem_text += f"Tag: {tag}\n\n"
	problem_text += f"Content: {content}\n\n"
	problem_text += f"Config: {config}\n\n"
	return problem_text

def parse_feedback_text(rdoc):
	score = rdoc['score']
	time = rdoc['time']
	memory = rdoc['memory']
	judge_texts = rdoc['judgeTexts']
	compiler_texts = rdoc['compilerTexts']
	test_cases = rdoc['testCases']
	feedback_text = f"Score: {score}\n"
	feedback_text += f"Time: {time}\n"
	feedback_text += f"Memory: {memory}\n\n"
	feedback_text += "Judges:\n"
	for judge_text in judge_texts:
		feedback_text += f"{judge_text}\n"
	feedback_text += "\n"
	feedback_text += "Compiler:\n"
	for compiler_text in compiler_texts:
		feedback_text += f"{compiler_text}\n"
	feedback_text += "\n"
	# "testCases": [
    #         {
    #             "id": 2,
    #             "subtaskId": 1,
    #             "status": 2,
    #             "score": 0,
    #             "time": 0,
    #             "memory": 6628,
    #             "message": "User output longer than standard answer."
    #         },
	feedback_text += "Test cases:\n"
	for test_case in test_cases:
		feedback_text += f"Test case {test_case['id']}: {test_case['message']}\n"
		feedback_text += f"Status: {code_to_status[test_case['status']]}\n"
		feedback_text += f"Score: {test_case['score']}\n"
		feedback_text += f"Time: {test_case['time']}\n"
		feedback_text += f"Memory: {test_case['memory']}\n\n"
	return feedback_text

In [26]:
import sys
import os
import copy
import time
sys.path.append(os.path.abspath(".."))
import llm_plan_bench as lpb
model = lpb.BlackboxLLM("ollama-qwen2.5-coder:32b")

In [23]:
model([{"role": "user","content":"hi my name is albert"}])

ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?


'Hi Albert! How can I assist you today?'

In [27]:
# for problem_id in range(102, 182):
for problem_id in range(103, 110):
	fetch_url = f"{web_ip}/p/{problem_id}"
	headers = {
	"Accept": "application/json"
	}
	problem_text = None
	try:
		fetch_response = session.get(fetch_url, headers=headers)
		if fetch_response.status_code != 200:
			print("Fetch problem failed:", fetch_response.text)
			exit()
		problem = fetch_response.json()
		problem_text = parse_problem_text(problem["pdoc"])
		json.dump(problem, open(f"problem-{problem_id}.json", "w"), ensure_ascii=False, indent=4)
	except Exception as e:
		print("Fetch problem failed:", e)
		continue
	print(problem_text)

	error_tolerance = 10
	messages = [
		{
			"role": "user",
			"content": problem_text
		}
	]
	# Chain of Thought, ask as user, ask the bot to generate any useful information, experience, tricks, knowledge related to solving this problem
	append_user_message(messages, copy.deepcopy(messages), "Please first tell me any related information, experience, tricks, knowledge which is useful for solving this problem. Please explain every knowledge in detail.")
	content = model(messages)
	append_assistant_message(messages, copy.deepcopy(messages), content)
	append_user_message(messages, copy.deepcopy(messages), "Now please explain your solution in detail, and generate the finalized code solution for this problem. I will upload the code to the online judge and then give you the feedback. Please format your response in a json file. The output must be in json format. The first key should be 'explanation', the second key should be 'lang' (compiler specified above in config), and the third key should be 'code'. The value should all be string. Please lang = cc.cc11. Thank you very much! You should generate a whole and complete program that can be compiled and run directly in code.")
	for trial_times in range(3):
		while True:
			content = model(messages)
			try:
				matches = json_pattern.findall(content)
				for match in matches:
					try:
						parsed_json = json.loads(match)
						break
					except Exception as e:
						print(e)
						continue
			except Exception as e:
				print(e)
				continue
			if "explanation" in parsed_json.keys() and "lang" in parsed_json.keys() and "code" in parsed_json.keys():
				break
		append_assistant_message(messages, copy.deepcopy(messages), content)
		lang = "cc.cc11"
		code = parsed_json["code"]
		submit_url = f"{web_ip}/p/{problem_id}/submit"
		submit_data = {
			"lang": lang,
			"code": code,
		}
		headers = {
			"Accept": "application/json"
		}
		submit_response = session.post(submit_url, json=submit_data, headers=headers)
		rid = None

		# Step 3: Check submission response
		if submit_response.status_code == 200:
			print("Submission successful")
			print("Response:", submit_response.json())
			rid = submit_response.json()["rid"]
			print("rid:",submit_response.json()["rid"])
		else:
			print("Submission failed")
			print("Status code:", submit_response.status_code)
			print("Response:", submit_response.text)
			# log
			continue
		# /record/:rid
		time.sleep(10)
		if rid is None:
			print("No record ID found, exiting...")
			# log
			continue
		# rid = "672836121431fba1f87ae7cc"  # Use the actual record ID
		result_url = f"{web_ip}/record/{rid}"
		headers = {
			"Accept": "application/json"
		}
		response = session.get(result_url, headers=headers)
		if response.status_code == 200:
			result = response.json()
		else:
			print("Failed to retrieve result:", response.status_code)
		feedback_text = parse_feedback_text(result["rdoc"])
		if result["rdoc"]["score"] == 100:
			print("Passed")
			break
		else:
			append_user_message(messages, copy.deepcopy(messages), feedback_text)
			append_user_message(messages, copy.deepcopy(messages), "Please explain the error in your code and try to fix it. Please format your response in a json file. The output must be in json format. The first key should be 'explanation', the second key should be 'lang' (compiler specified above in config), and the third key should be 'code'. The value should all be string. Please lang = cc.cc11. Thank you very much! You should generate a whole and complete program that can be compiled and run directly in code.")
			continue
	# log
	print("Finish problem", problem_id)
	json.dump({
		"problem_id": problem_id,
		"messages": messages,
	}, open(f"problem-{problem_id}-messages.json", "w"), ensure_ascii=False, indent=4)
	

ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?


Please solve the following problem:

Title: A Pivotal Question
Tag: ['ICPC GNY Regional 2023']

Content: {"en":"\n\n### Problem B A Pivotal Question\n\n**Time limit: 3 seconds**\n\nQuicksort is a recursive sorting algorithm developed in 1959 by Tony Hoare. One of the major steps in the algorithm is the partition step: given an element p in the array (the pivot element) rearrange the elements in the array as shown below where all the values in XL are ≤ p and all elements in XR are > p.\n\n\\[\np \\quad XL \\quad XR\n\\]\n\nFigure B.1 below shows an array before and after it’s been partitioned with the pivot element 13. Note that the elements in XL and XR are typically not in sorted order and either one of them could be empty.\n\n13 94 99 1 53 20 63 8 79 11 - 1 11 8 13 53 20 63 99 79 94\n\n**Figure B.1:** An array before and after a partition\n\nHow a partition is executed and how a pivot element is selected are fascinating questions but are not of interest to us. What we would like you 

ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?


Submission successful
Response: {'rid': '67474c81545795241003fa1b', 'url': '/record/67474c81545795241003fa1b', 'UiContext': {'cdn_prefix': '/', 'url_prefix': '/', 'ws_prefix': '/', 'constantVersion': '3b7146a4', 'domainId': 'system', 'domain': {'_id': 'system', 'lower': 'system', 'owner': 1, 'name': 'Hydro', 'bulletin': '### 欢迎来到 Hydro ！  \n\n当你看到这行字时，说明 Hydro 的基础功能已经正常运行。  \n下一步您应该：\n\n- 在右上角注册一个账号。  \n- 回到刚刚的终端，使用 `hydrooj cli user setSuperAdmin 2` 将 UID 为 2 的用户设置为超级管理员。  \n- 使用 `pm2 restart hydrooj` 重启以使管理员更改立刻生效。  \n- 前往 “题库” 面板，查看创建的示例题目是否正常工作。  \n- 使用超级管理员账号登录，通读 [控制面板 -> 系统设置](/manage/setting) 页面，按需配置。  \n\n如果您需要可以直接导入的题目，可以 [加入 Hydro 用户群](https://jq.qq.com/?_wv=1027&k=pKYrk4yp) 或是 [从 Hydro 题库下载](https://hydro.ac/d/tk/p) 。  \n您也可以下载 [一本通编程启蒙](https://hydro.ac/ybtbas.zip) 和 [深入浅出程序设计竞赛（基础篇）](https://hydro.ac/srqc.zip) 题库并按照压缩包内说明导入系统。  \n如果您需要题目配置指南，可以 [查看文档](https://hydro.js.org) 或是 [查看配置示例](https://hydro.ac/d/system_test/) 。\n\n当你已经熟悉本系统的操作，可以在 [管理域 > 编辑域资料](/domain/edit) 页面修改这

ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?


...

Finish problem 105
Please solve the following problem:

Title: Three Spheres and a Tetrahedron
Tag: ['ICPC GNY Regional 2023']

Content: {"en":"\n\n### Problem E Three Spheres and a Tetrahedron\n\n**Time limit: 2 seconds**\n\nGiven a tetrahedron \\( OABC \\) with vertices \\( O, A, B \\) and \\( C \\).\n\nThere is a sphere, \\( S_1 \\) (red, center \\( Q_1 \\)), inscribed in the tetrahedron tangent to the inside of each face \\( OAB \\) (gray), \\( OAC \\) (brown), \\( OBC \\) (magenta) and \\( ABC \\) (cyan and black).\n\nThere is a second sphere, \\( S_2 \\) (green, center \\( Q_2 \\)), tangent to the (extended) inside of \\( OAB \\), \\( OAC \\) and \\( OBC \\) and to the outside of \\( ABC \\). (There is actually such a sphere for each face, tangent to the outside of the face and the inside of the other extended faces).\n\nThere is a third larger sphere, \\( S_3 \\) (blue, center \\( Q_3 \\)), which passes thru vertices \\( A, B \\) and \\( C \\) and is tangent to each of \\( S_1 \

ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?


Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Submission successful
Response: {'rid': '67474e4b545795241003fafa', 'url': '/record/67474e4b545795241003fafa', 'UiContext': {'cdn_prefix': '/', 'url_prefix': '/', 'ws_prefix': '/', 'constantVersion': '3b7146a4', 'domainId': 'system', 'domain': {'_id': 'system', 'lower': 'system', 'owner': 1, 'name': 'Hydro', 'bulletin': '### 欢迎来到 Hydro ！  \n\n当你看到这行字时，说明 Hydro 的基础功能已经正常运行。  \n下一步您应该：\n\n- 在右上角注册一个账号。  \n- 回到刚刚的终端，使用 `hydrooj cli user setSuperAdmin 2` 将 UID 为 2 的用户

ic| Error: Failed to access the underlying source code for analysis. Was ic() invoked in a REPL (e.g. from the command line), a frozen application (e.g. packaged with PyInstaller), or did the underlying source code change during execution?


KeyboardInterrupt: 